In [1]:
# import libraries
import arcpy, os, sys
# check out spatial analyst extension
arcpy.CheckOutExtension("SPATIAL")

'CheckedOut'

In [2]:
# function to copy output files to disk
def copyToDisk(inFile,outFile):
    if arcpy.Exists(outFile):
        return((1,"The file: "+outFile+" already exists"))
    else:
        try:
            try:
                arcpy.CopyRaster_management(inFile,outFile,"#","#",65535,"#","#","16_BIT_UNSIGNED")
            except:
                inFile.save(outFile)
            return((1,"Successfully created "+outFile))
        except:
            return((0,arcpy.GetMessages()))

In [3]:
# define the LECZ code
leczCode=10
landCode=31
# define input variables
inputCoastlinePoly= r'data\inputs\lka_admin0_line.shp'# path to some line layer representing the coastline for your AOI
inputLandMask=r'data\inputs\lka_admin0.tif'# path to some file which represents all of the land areas in your AOI
inputDEM= r'data\inputs\lka_merit_raw.tif'# path to the DEM data that will be used to create the LECZs
# please note that inputLandMask and inputDEM should be in the same extent and cellsize
# check if the files exist
if not arcpy.Exists(inputCoastlinePoly):print("The input file is missing: "+inputAdminPoly)
if not arcpy.Exists(inputLandMask):print("The input file is missing: "+inputLandMask)
if not arcpy.Exists(inputDEM):print("The input file is missing: "+inputDEM)
# set the processing environments to the extent and cellsize to that of the inputDEM
arcpy.env.extent=inputDEM
arcpy.env.cellSize=inputDEM

In [4]:
# step 1: extract the input DEM to within the inputLandMask and
# reclassify the input DEM into less than or equal to some lecz
try:    
    rec=arcpy.sa.ExtractByMask(arcpy.sa.Con(arcpy.Raster(inputDEM)>leczCode,landCode,leczCode),inputLandMask)
except:
    print(arcpy.GetMessages())
# copy the output to disk
outRec=r'data\outputs\intermediate\lka_merit_reclass_'+str(leczCode)+'.tif'
copyResults=copyToDisk(rec,outRec)
if copyResults[0]==0:
    sys.exit(copyResults[1])
else:
    print(copyResults[1])

Successfully created data\outputs\intermediate\lka_merit_reclass_10.tif


In [5]:
# step 2: region group to identify contiguous regions, but only those areas that are not coded land (31)
# build an attribute table on the integer raster that results
try:
    rg=arcpy.sa.RegionGroup(arcpy.sa.SetNull(arcpy.Raster(rec)==landCode,leczCode),"EIGHT","WITHIN")
    arcpy.BuildRasterAttributeTable_management(rg)
except:
    print(arcpy.GetMessages())
# copy the output to disk    
outRg=r'data\outputs\intermediate\lka_merit_region_group_'+str(leczCode)+'.tif'
copyResults=copyToDisk(rg,outRg)
if copyResults[0]==0:
    sys.exit(copyResults[1])
else:
    print(copyResults[1])

Successfully created data\outputs\intermediate\lka_merit_region_group_10.tif


In [6]:
# step 3: convert the region group raster to polygon and select by location 
# those areas which intersect the coastline
outRgPoly=r'data\outputs\intermediate\lka_merit_region_group_poly_'+str(leczCode)+'.shp'
if arcpy.Exists(outRgPoly):
    print("The output polygon "+outRgPoly+" already exists")
else:
    try:
        arcpy.RasterToPolygon_conversion(rg,outRgPoly,"NO_SIMPLIFY","VALUE")
    except:
        print(arcpy.GetMessages())
try:
    # ESRI selection tools require layers as input
    targetLyr="inputcoastlinepolylyr"
    arcpy.MakeFeatureLayer_management(inputCoastlinePoly,targetLyr)
    selectLyr="rgPolylyr"
    arcpy.MakeFeatureLayer_management(outRgPoly,selectLyr)
    arcpy.SelectLayerByLocation_management(selectLyr,"INTERSECT",targetLyr)
    selectedPolys=r'data\outputs\intermediate\lka_merit_region_group_poly_selection_'+str(leczCode)+'.shp'
    if arcpy.Exists(selectedPolys):
        print("The output polygon "+selectedPolys+" already exists")
    else:
        arcpy.CopyFeatures_management(selectLyr,selectedPolys)
        arcpy.AddField_management(selectedPolys,"lecz","Short")
        arcpy.CalculateField_management(selectedPolys,"lecz",leczCode,"PYTHON")
except:
    print(arcpy.GetMessages())
        
    

In [7]:
# step 4: rasterize the selected region grouped polygons
outRgRaster=r'data\outputs\intermediate\lka_merit_lecz_'+str(leczCode)+'.tif'
if arcpy.Exists(outRgRaster):
        print("The file: "+outRgRaster+" already exists")
else:
    try:        
        arcpy.PolygonToRaster_conversion(selectedPolys,"lecz",outRgRaster)            
    except:
        print(arcpy.GetMessages())

In [8]:
# step 5: finally combine the LECZ with land area values
outLECZRaster=r'data\outputs\lka_merit_lecz_'+str(leczCode)+'.tif'
if arcpy.Exists(outLECZRaster):
        print("The file: "+outLECZRaster+" already exists")
else:
    try:  
        # if the outRgRaster is null, but outRec is not, code as land (31)
        # otherwise code as the leczCode
        leczFinal=arcpy.sa.Con(arcpy.sa.IsNull(outRgRaster)==1,
                               arcpy.sa.Con(arcpy.sa.IsNull(outRec)==0,31),
                               leczCode)            
    except:
        print(arcpy.GetMessages())
# copy the output to disk
copyResults=copyToDisk(leczFinal,outLECZRaster)
if copyResults[0]==0:
    sys.exit(copyResults[1])
else:
    print(copyResults[1])

Successfully created data\outputs\lka_merit_lecz_10.tif
